# Modelado / Modelling 

Un **modelo** es una representación simplificada de la realidad creada para un propósito concreto. Las hipótesis que asumimos, la selección de características relevantes, las restricciones y la trazabilidad de cómo se construye forman parte de ese modelo.

Un ejemplo clásico es un **mapa**: una representación cartográfica que permite situar lugares de interés, calcular distancias o analizar relaciones topológicas entre puntos.

<img src="images/Netzentwuerfe.png" width="30%"/>

Fuente imagen: [Wikipedia](https://es.wikipedia.org/wiki/Proyecci%C3%B3n_cartogr%C3%A1fica)


En **ciencia de datos** distinguimos, de forma general:

- **Modelos predictivos**: conjuntos de fórmulas o reglas que permiten estimar valores desconocidos (por ejemplo, predecir ventas futuras).  
- **Modelos descriptivos**: ayudan a descubrir patrones subyacentes o estructura en los datos (por ejemplo, segmentos de clientes).

Los datos usados por el modelo representan hechos u observaciones de la realidad (o de otro modelo). Siguiendo el ejemplo del mapa, serían los puntos, contornos o bordes del territorio representado.

<img src="images/modeloDatos.png" width="50%"/>


> Fuente de ambas figuras [2]


## El proceso de aprendizaje automático

Podemos esquematizar el proceso de aprendizaje automático en varias etapas:

1. **Preparación de datos**  
   Carga, limpieza, transformación y división del conjunto de datos.  
   - Incluye el análisis exploratorio de datos (EDA) y la selección de características.

2. **Selección de la técnica**  
   Elección del tipo de tarea (clasificación, regresión, clustering…), de los modelos candidatos y de las restricciones prácticas (tiempo, interpretabilidad, recursos, etc.).

3. **Ajuste de hiperparámetros**  
   Los hiperparámetros son configuraciones del modelo que no se aprenden directamente de los datos (por ejemplo, profundidad máxima de un árbol) y deben ajustarse manualmente o mediante búsqueda automática.

4. **Evaluación del modelo**  
   Pruebas iniciales, elección de métricas, ajustes iterativos y evaluación final sobre datos no vistos.


### Selección de atributos o métricas adecuadas

Dado un conjunto de muestras con muchas características, el reto es **elegir aquellas que realmente contribuyen al aprendizaje del modelo**.

Supongamos el siguiente ejemplo de personas:
<img src="images/people.png" width="50%"/>

Las características disponibles son:

- Forma de la cabeza: cuadrada o circular.  
- Forma del cuerpo: rectangular o ovalada.  
- Color del cuerpo: negro o blanco.  
- Función de su compra: *yes/no*.

Esta última característica es el atributo objetivo (*target*): lo que queremos predecir en futuros clientes.

Para predecirlo, debemos preguntarnos qué atributos aportan información útil. Si solo usáramos, por ejemplo, el color del cuerpo, podríamos perder variabilidad importante del resto de la población y construir un modelo pobre.

Más adelante retomaremos este problema de selección de atributos introduciendo los conceptos de **entropía** y **ganancia de información**.



## Nuestra primera aplicación de machine learning (ML)

### Instalación de librerías

La librería de Python que implementa múltiples algoritmos de ML es **scikit-learn**

- Scikit-learn: https://scikit-learn.org/stable/

Trabajemos con las versiones de librerías usadas en Google Colab (09/12/25) como sistema de referencia.


Podemos usar UV:

In [89]:
!uv add scikit-learn==1.6.1
!uv add sklearn-pandas==2.2.0
!uv add seaborn==0.13.2

Resolved 99 packages in 5ms
Audited 94 packages in 0.20ms
Resolved 99 packages in 0.66ms
Audited 94 packages in 0.01ms
Resolved 99 packages in 0.64ms
Audited 94 packages in 0.01ms


Y si tuvieramos pip:

```bash
!pip install scikit-learn==1.6.1
!pip install sklearn-pandas==2.2.0
!pip install seaborn==0.13.2
```

### Datos

Vamos a utilizar el siguiente catálogo de datos: **Mushroom Data Set** http://archive.ics.uci.edu/ml/datasets/Mushroom

**Disponible** en: "data/mushrooms.csv"

In [90]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/mushrooms.csv")

# Por simplicidad, renombramos las columnas
es_col = [
    "clase",
    "forma del sombrero",
    "superficie del sombrero",
    "color del sombrero",
    "magulladuras",
    "olor",
    "unión de las láminas",
    "espaciamiento de las láminas",
    "tamaño de las láminas",
    "color de las láminas",
    "forma del tallo",
    "raíz del tallo",
    "superficie del tallo por encima del anillo",
    "superficie del tallo por debajo del anillo",
    "color del tallo por encima del anillo",
    "color del tallo por debajo del anillo",
    "tipo de velo",
    "color del velo",
    "número de anillos",
    "tipo de anillo",
    "color de la impresión de esporas",
    "población",
    "hábitat"
]
df.columns = es_col

print(df.shape)
print("-"*100)
print(df.columns)
print("-"*100)
print(df.head(3))

(8124, 23)
----------------------------------------------------------------------------------------------------
Index(['clase', 'forma del sombrero', 'superficie del sombrero',
       'color del sombrero', 'magulladuras', 'olor', 'unión de las láminas',
       'espaciamiento de las láminas', 'tamaño de las láminas',
       'color de las láminas', 'forma del tallo', 'raíz del tallo',
       'superficie del tallo por encima del anillo',
       'superficie del tallo por debajo del anillo',
       'color del tallo por encima del anillo',
       'color del tallo por debajo del anillo', 'tipo de velo',
       'color del velo', 'número de anillos', 'tipo de anillo',
       'color de la impresión de esporas', 'población', 'hábitat'],
      dtype='object')
----------------------------------------------------------------------------------------------------
  clase forma del sombrero superficie del sombrero color del sombrero  \
0     p                  x                       s                  

### Planteamiento del objetivo de ML

¿Cuál es el objetivo que planteamos? ¿Qué valos nos interesa encontrar?

En este caso, **queremos determinar la población según las características del hongo**

Esta variable contiene los siguientes datos:
- population: abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y


In [91]:
# Podemos ver los valores de la variable objetivo
print(df["población"].head()) 
value,counts = np.unique(df["población"], return_counts=True)
print(value,counts)


0    s
1    n
2    n
3    s
4    a
Name: población, dtype: object
['a' 'c' 'n' 's' 'v' 'y'] [ 384  340  400 1248 4040 1712]


### ¿Qué tipo de poblema de ML es?
- Supervisado o No Supervisado?
- Clasificación, Regresión o Agrupamiento?


### Selección de características (feature selection) 

En este primer problema, usaremos todas las muestras. 

Tenemos que separar el dataset en dos partes: características y variable objetivo.

In [92]:
df_y = df["población"].copy()
df_x = df.drop(labels=["población"],axis=1).copy()

print(df_x.shape)
print(df_y.shape)

(8124, 22)
(8124,)


### Preparación de los datos de entreno (train) y datos de comprobación (test)

Podemos hacerlo de múltiples maneras, y es crítico en series temporales. Hay métodos automaticos que nos separan las muestras como por ejemplo: ```train_test_split```


In [93]:
from sklearn.model_selection import train_test_split

x_train, x_test,  y_train, y_test = train_test_split(df_x,df_y, test_size=0.2, random_state = 0)
print("x_train:", x_train.shape)
print("y_train: ",y_train.shape)
print("-"*100)
print("x_test: ",x_test.shape)
print("y_test: ",y_test.shape)


x_train: (6499, 22)
y_train:  (6499,)
----------------------------------------------------------------------------------------------------
x_test:  (1625, 22)
y_test:  (1625,)


### Elección del algoritmo

Existen múltiples algoritmos según el tipo de problema de ML. En este ejemplo, usaremos ```Máquinas de Vectores de Soporte (SVM)'''
- https://scikit-learn.org/stable/modules/svm.html
- https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html


In [94]:
from sklearn.svm import SVC

clf = SVC(C=1.0, kernel="linear", random_state=0) #¿Que implica el kernel?
#clf.fit(x_train, y_train) # Alerta: Genera un error!!! ¿Por qué?


In [95]:
# El error proviene porque los SVM tan solo funcionan con variables continuas (números)
# Algunos algoritmos como los árboles de decisión, si funcionan con variables categoricas
# En nuestro caso, tenemos que transformar las variables categoricas a variables discretas

# Existen múltiples maneras de hacerlo. En este caso, usaremos el ```LabelEncoder```
# Basicamente, lo que hace es asignar un número a cada categoría. La gestión de esta asignación recae en este encoder.
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() # lo creamos

# Aplicamos el encoder a todas las columnas del dataset inicial!!! 
for i in df.columns:
    df[i] = le.fit_transform(df[i]) # lo aplicamos a cada columna, 

print(df.iloc[:3,:3]) #imprimimos un par de filas y columnas para ver el resultado

# Por lo tanto, no nos quedará más remedio que crear de nuevo la partición del dataset con la variable objetivo y las características 
x_train, x_test,  y_train, y_test  = train_test_split(df.drop('población', axis=1),df['población'], test_size=0.2,random_state=0)

   clase  forma del sombrero  superficie del sombrero
0      1                   5                        2
1      0                   5                        2
2      0                   0                        2


In [96]:
clf.fit(x_train, y_train) # Ahora sí! Entrenamos el modelo: TRAIN

SVC(kernel='linear', random_state=0)

In [97]:
# Una vez entrenado, podemos usar el modelo para predecir sobre nuevos datos o sobre los datos de control
y_pred = clf.predict(x_test) # predecimos sobre los valores reservados

### Métricas

Los datos de comprobación o de test nos sirven para medir la bondad del algoritmo sobre valores objetivos que conocemos.

Las métricas dependerán del tipo de algoritmo. En este caso, estamos aplicando un algoritmo de clasificación, por lo tanto, tenemos métricas como la precisión, recall, f1-score, etc.

In [98]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred)) #comprobamos el error

              precision    recall  f1-score   support

           0       0.44      0.58      0.50        65
           1       0.67      1.00      0.81        62
           2       0.48      0.33      0.39        93
           3       0.45      0.42      0.44       237
           4       0.79      0.63      0.70       827
           5       0.48      0.70      0.57       341

    accuracy                           0.61      1625
   macro avg       0.55      0.61      0.57      1625
weighted avg       0.64      0.61      0.62      1625



Cuando se obtienen los valores, el problema continua: ¿Qué interpretación podemos hacer? ¿Es un buen método?...

## Evaluación de un modelo de aprendizaje

La evaluación tiene como propósito **validar** el modelo y **ganar confianza** en su uso.

Podemos distinguir dos perspectivas:

- **Cualitativa**  
  ¿El modelo contribuye a los objetivos de negocio y a la toma de decisiones?  
  ¿Es integrable en los sistemas existentes?  
  ¿Encaja con el *stack* tecnológico y los requisitos legales/éticos?

- **Cuantitativa**  
  Se apoya en métricas: tanto de rendimiento computacional (tiempo, memoria, coste) como de desempeño del modelo (error, precisión, etc.).


### Métricas

Las métricas concretas dependen del tipo de problema.  
En problemas de **clasificación** y **regresión** se utilizan, entre otras, las siguientes.

#### En clasificación
  
<img src="images/TypeErrors.png" width="60%"/>

> Source: NillsF blog

- **Accuracy**: proporción de predicciones correctas sobre el total:   $ \frac{TP+TN}{Total}.$
- **Precision**: proporción de predicciones positivas que son realmente positivas:   $\frac{TP}{Results} = \frac{TP}{TP+FP}$
- **Recall** (sensibilidad): proporción de positivos reales correctamente identificados:   $\frac{TP}{Predictive Results} = \frac{TP}{TP+FN}$
- **F1-score**: media armónica de precisión y *recall*:  $\frac{2}{recall^{-1}+precision^{-1}}$
- **Área bajo la curva ROC (AUC)**: para clasificación binaria.  
  La curva ROC compara la tasa de verdaderos positivos frente a la tasa de falsos positivos para distintos umbrales, y el AUC mide la capacidad del modelo para separar ambas clases.
  
- **Matriz de confusión**: tabla que cruza valores reales con valores predichos:  
$$\begin{equation}
\begin{pmatrix}
TP & FN \\
FP & TN 
\end{pmatrix}
\end{equation}$$

#### En regresión

- **Mean Absolute Error (MAE)**: media de las diferencias absolutas entre predicciones y valores reales.

- **Mean Squared Error (MSE)**: media de los cuadrados de las diferencias entre predicciones y valores reales (no es exactamente la desviación estándar, sino su cuadrado excepto por factores de normalización).

- **Coeficiente de determinación \(R^2\)**: indica qué proporción de la variabilidad de la variable objetivo explica el modelo.

- **\(R^2\) ajustado (Adjusted \(R^2\))**: versión de \(R^2\) que penaliza la inclusión de variables adicionales y es más adecuada para comparar modelos con distinto número de predictores.

Muchas de estas métricas (y otras adicionales) están implementadas en la librería **scikit-learn**:
- https://scikit-learn.org/stable/modules/model_evaluation.html

> Nota: Iremos utilizando las principales métricas en las unidades siguientes.


Para el caso anterior, podemos calcular algunas de estas métricas de la siguiente manera:

In [99]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.6123076923076923

In [100]:
from sklearn import metrics

metrics.precision_score(y_test,y_pred,average="weighted")

0.6413835956221657

In [101]:
print("Recall: ",metrics.recall_score(y_test,y_pred,average="weighted"))
print("F1_score: ",metrics.f1_score(y_test,y_pred,average="weighted"))

Recall:  0.6123076923076923
F1_score:  0.6153073145466426


## Rendimiento computacional

Cada operación tiene un **coste computacional**: no solo tiempo de ejecución, sino también **memoria** para almacenar datos y valores intermedios.

La capacidad de CPU/GPU suele medirse en operaciones por segundo, por ejemplo:

- MIPS (millones de instrucciones por segundo)  
- MFLOPS / GFLOPS (millones / miles de millones de operaciones en coma flotante por segundo)
  
Este rendimiento influye directamente en el **tiempo de entrenamiento** y de **predicción** de los modelos.

El **lenguaje de programación**, su **compilador/intérprete** y las **librerías numéricas** condicionan cómo se aprovechan CPU y GPU: número de *cores* utilizados, organización de tareas, gestión de memoria, etc.


La **complejidad del modelo**, el **tamaño de los datos** y su **representación** afectan al rendimiento y, por tanto, a su aplicabilidad en un entorno real, donde el tiempo de respuesta debe ser razonable.  
Ejemplo: un sistema de recomendación cuyos resultados se calculan dinámicamente al cargar una página web.

- https://scikit-learn.org/stable/computing/computational_performance.html


In [102]:
clf.fit(x_train, y_train) 

SVC(kernel='linear', random_state=0)

In [103]:
from time import perf_counter

t0 = perf_counter()
clf.fit(x_train, y_train) 
t1 = perf_counter()
print("Tiempo de entrenamiento:", t1 - t0, "s")

t0 = perf_counter()
y_pred = clf.predict(x_test)
t1 = perf_counter()
print("Tiempo de predicción:", t1 - t0, "s")


Tiempo de entrenamiento: 0.48070470802485943 s
Tiempo de predicción: 0.11679345811717212 s


In [104]:
!uv add memory_profiler

Resolved 99 packages in 0.48ms
Audited 94 packages in 0.01ms


In [105]:
from memory_profiler import memory_usage

def train():
    clf.fit(x_train, y_train)
    return clf

mem_usage = memory_usage(train)
print("Memoria máxima (MB):", max(mem_usage))

Memoria máxima (MB): 242.875


La memoria no es un problema a priori que debe de preocuparos en este punto de vuestro aprendizaje. **El tiempo de entrenamiento y predicción, sí.**

[![License: CC BY 4.0](https://img.shields.io/badge/License-CC_BY_4.0-lightgrey.svg)](https://creativecommons.org/licenses/by/4.0/) <br/>
Isaac Lera and Gabriel Moya <br/>
Universitat de les Illes Balears <br/>
isaac.lera@uib.edu, gabriel.moya@uib.edu